In [1]:
import pandas as pd
import numpy as np
import shutil
from pathlib import Path

DATA_DIR = Path("data")
INPUT_PATH = DATA_DIR / "books.csv"
BACKUP_PATH = DATA_DIR / "books.csv.bak"

if not INPUT_PATH.exists():
    DATA_DIR.mkdir(exist_ok=True)
    pd.DataFrame({
        'book_id': [1, 2, 3],
        'avg_rating': [4.5, 3.2, 4.8],
        '5_star': [10, 5, 20],
        'similar_books': ["2:0.5|3:0.9", "99:0.1", np.nan]
    }).to_csv(INPUT_PATH, index=False)

shutil.copy(INPUT_PATH, BACKUP_PATH)
df = pd.read_csv(INPUT_PATH, on_bad_lines='skip')
df = df.dropna(subset=['avg_rating'])

star_cols = [col for col in df.columns if col.endswith('star')]
df['rating_count'] = df[star_cols].sum(axis=1)

def calculate_score(avg_rating, rating_count):
    return avg_rating - avg_rating / np.log(rating_count + 10)

valid_ids_set = set(df['book_id'].astype(str))

def parse_row(encoded_str):
    if not isinstance(encoded_str, str) or not encoded_str:
        return [], 0.0
    
    parsed_ids = [item.split(":")[0] for item in encoded_str.split("|")]
    total = len(parsed_ids)
    
    if total == 0:
        return [], 0.0
        
    found_count = sum(pid in valid_ids_set for pid in parsed_ids)
    
    return parsed_ids, (found_count / total) * 100

results = [parse_row(x) for x in df['similar_books']]

df['sim_lists'], df['exist_percent'] = zip(*results)
# df_clean = df[df['exist_percent'] == 100].copy()
# print(len(test))
# test = test.explode('sim_lists').drop(columns=['exist_percent'])
# test 


In [ ]:

# 1. Filter: Keep only the rows where 100% of links exist (the 1268 rows)
df_clean = df[df['exist_percent'] == 100].copy()

# 2. Build stats lookup from the FULL original df (so we can find stats for referenced books)
stats = df.set_index(df['book_id'].astype(str))[['avg_rating', 'rating_count']].to_dict('index')

# 3. Formatting function: Converts "ID:Score" -> "ID:Avg:Count"
def format_sims(encoded_str):
    if not encoded_str or pd.isna(encoded_str): return ""
    
    parts = []
    for item in encoded_str.split('|'):
        bid = item.split(':')[0]
        # We know bid exists in stats because exist_percent is 100
        s = stats[bid] 
        parts.append(f"{bid}:{s['avg_rating']}:{int(s['rating_count'])}")
    return "|".join(parts)

# 4. Apply format
df_clean['similar_books'] = df_clean['similar_books'].apply(format_sims)

# 5. Drop helper columns and Save
keep_cols = [c for c in df_clean.columns if c not in ['sim_lists', 'exist_percent', 'rating_count']]
df_clean[keep_cols].to_csv(INPUT_PATH, index=False)


In [ ]:
453706:4.7:1906|6082928:4.59:2326|1842376:4.69:1842|251910:4.7:1891|4481992:4.63:1779|251909:4.67:1937|6381255:4.59:1770|13557:4.6:2057|8470939:4.62:1651|3650022:4.59:2127|1982828:4.69:1827|1324795:4.6:2179|7749279:4.64:1669|251911:4.62:1910|1842377:4.53:2081|251913:4.66:1998|1324783:4.57:2226|6082947:4.62:2297|7131571:4.54:1707|8853350:4.63:1627

In [ ]:
stats_map = {}
for _, row in df.iterrows():
    try:
        bid = int(row['book_id'])
        # Sum columns 1_star through 5_star
        total_ratings = sum(int(row.get(f"{i}_star", 0)) for i in range(1, 6))
        avg_rating = float(row.get('avg_rating', 0.0))
        stats_map[bid] = (avg_rating, total_ratings)
    except (ValueError, KeyError):
        continue

# # 4. Process Rows
# rows_to_keep = []
# dropped_count = 0
# fixed_cells = 0

# print("Processing rows...")
# for _, row in df.iterrows():
#     original_sims = row.get('similar_books')
    
#     # If empty/NaN, keep the row
#     if pd.isna(original_sims) or original_sims == "":
#         rows_to_keep.append(row)
#         continue

#     new_sims_list = []
#     keep_row = True
    
#     # Check every book in the similar_books string
#     for item in str(original_sims).split('|'):
#         parts = item.split(':')
        
#         if len(parts) == 3:
#             # Format is already Correct: ID:AVG:COUNT
#             new_sims_list.append(item)
#         elif len(parts) == 2:
#             # Format is Broken/Old: ID:SCORE
#             neighbor_id = int(parts[0])
            
#             if neighbor_id in stats_map:
#                 # REPAIR: We have the raw stats in our lookup map
#                 n_avg, n_count = stats_map[neighbor_id]
#                 new_sims_list.append(f"{neighbor_id}:{n_avg}:{n_count}")
#                 fixed_cells += 1
#             else:
#                 # FATAL: We don't have stats for this neighbor. 
#                 # We must drop this parent row so the scraper fetches it again.
#                 keep_row = False
#                 break
    
#     if keep_row:
#         row['similar_books'] = "|".join(new_sims_list)
#         rows_to_keep.append(row)
#     else:
#         dropped_count += 1

# # 5. Save Result
# field_names = [
#     "book_id", "title", "authors", "avg_rating", "review_count", 
#     "num_pages", "lang", "1_star", "2_star", "3_star", "4_star", 
#     "5_star", "genres", "series", "year", "description", "similar_books"
# ]

# cleaned_df = pd.DataFrame(rows_to_keep)
# cleaned_df.to_csv(INPUT_PATH, index=False, columns=field_names)

,book_id,title,authors,avg_rating,review_count,num_pages,lang,1_star,2_star,3_star,4_star,5_star,genres,series,year,description,similar_books
0,3,Harry Potter and the Sorcerer's Stone (Harry P...,J.K. Rowling,4.47,191094.0,309.0,English,189284,215497,963005,2659719,7272601,Fantasy|Fiction|Young Adult|Harry Potter|Magic...,45175-harry-potter,1997,"""Turning the envelope over, his hand trembling...",2767052:4.35|41865:3.67|5907:4.3|25856606:4.12...
1,21,A Short History of Nearly Everything,Bill Bryson,4.22,17350.0,544.0,English,8585,13967,55790,140850,200588,Nonfiction|Science|History|Audiobook|Humor|Phy...,NaN,2003,Bill Bryson describes himself as a reluctant t...,3869:4.21|61535:4.16|23692271:4.34|1842:4.04|3...
2,350,Stranger in a Strange Land,Robert A. Heinlein|James Warhola,3.92,11013.0,525.0,English,13601,23637,64048,99189,125413,Science Fiction|Fiction|Classics|Science Ficti...,NaN,1961,"A human raised on Mars, Valentine Michael Smit...",29579:4.17|375802:4.31|70535:4.18|41804:4.21|6...
3,327,Why Zebras Don't Get Ulcers,Robert M. Sapolsky,4.17,1516.0,560.0,NaN,161,611,2742,6205,7348,Psychology|Nonfiction|Science|Health|Biology|S...,NaN,1993,"Now in a third edition, Robert M. Sapolsky's a...",216151199:4.28|23719305:4.09|64582:4.02|288204...
4,421,The White Album,Joan Didion,4.02,5743.0,224.0,English,395,1965,10052,20645,16381,Nonfiction|Essays|Memoir|Classics|History|Jour...,NaN,1979,"First published in 1979, Joan Didion's The Whi...",206318561:4.08|2687519:3.89|207293782:3.24|475...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21818,43393197,The Homesteader's Sweetheart (Wind River Heart...,Lacy Williams,4.48,193.0,298.0,English,11,32,239,735,1600,Romance|Historical Romance|Historical|Westerns...,364576-wind-river-hearts,2012,"Pursued by an unwelcome suitor, banker's daugh...",232475701:3.88|232475716:3.88|217124245:3.86|2...
21819,44024246,"The Legend of Korra: Ruins of the Empire, Part...",Michael Dante DiMartino|Michelle Wong|Killian Ng,4.47,184.0,80.0,English,14,45,344,1253,2334,Comics|Graphic Novels|Fantasy|LGBT|Young Adult...,244939-the-legend-of-korra-comics-ruins-of-the...,2019,What will it take to stop a war?\n\nKing Wu's ...,39005056:3.77|41161043:3.84|43845605:3.79|2396...
21820,43594047,"NEW, CLEAR VISION: Nuclear Energy: friend or f...",Mike Scantlebury,5.00,12.0,243.0,English,0,0,0,0,12,NaN,NaN,2018,"Who wants nuclear power now? Well, the governm...",235363928:3.98|231247032:3.99|233845008:3.68|2...
21821,44168344,Tout le bleu du ciel,Mélissa Da Costa,4.46,7116.0,645.0,French,393,1185,4790,14302,32868,Fiction|Roman|France|Romance|Contemporary|Audi...,NaN,2019,"Petiteannonce. fr : Émile, 26 ans, condamné à ...",214211465:3.58|218395095:3.89|57280590:3.57|20...


In [ ]:
# main
this_month = datetime.now().strftime('%m-%Y')
this_day = datetime.now().strftime('%d-%m-%Y')
new_export_path = f'data/{this_day}_goodreads_library_export.csv'
this_months_scrape_path = f'data/{this_month}_goodreads_scraped.csv'

while True:
    book_ids, books_to_scrape = check_mybooks(new_export_path, this_months_scrape_path)
    if books_to_scrape:
        for i, book_id in enumerate(books_to_scrape):
            try:
                print(f'\nScraping book-id:{book_id} ({i+1}/{len(books_to_scrape)})')
                start = datetime.now()
                book = scrape_book(book_id)
                if book:
                    json.dump(book, open(f'metadata/{book_id}_metadata.json', 'w'))
                else:
                    print(f'    scrape_book() returned empty')
                print(f'{datetime.now() - start}')
            except Exception as e:
                print(e)

        books = condense_books('metadata')
        book_df = pd.DataFrame(books)
        
        if os.path.isfile(this_months_scrape_path):
            old_df = pd.read_csv(this_months_scrape_path)
            book_df = pd.concat([old_df, book_df])
            book_df = book_df[book_df['book_id'].isin(book_ids)]
            book_df = book_df.drop_duplicates(subset=[col for col in book_df.columns if col != 'genres'])

        book_df.to_csv(this_months_scrape_path, index=False, encoding='utf-8')
    else:
        # delete_metadata()
        print('ALL BOOKS HAVE BEEN SCRAPED')
        break

In [ ]:
goodreads_export = pd.read_csv(new_export_path)
# goodreads_export = pd.read_csv('data/20-01-2025_goodreads_library_export.csv')
goodreads_export['Original Publication Year'] = goodreads_export['Original Publication Year'].fillna(goodreads_export['Year Published'])
goodreads_export = goodreads_export[['Book Id', 'Author', 'My Rating', 'Number of Pages', 'Original Publication Year']]
goodreads_export = goodreads_export.rename(columns={'Book Id':'book_id',
                                                    'Author': 'author',
                                                    'My Rating': 'my_rating',
                                                    'Number of Pages': 'num_pages',
                                                    'Original Publication Year': 'year'})
threshold = (goodreads_export['num_pages'].mean() - goodreads_export['num_pages'].std())
goodreads_export.loc[goodreads_export['num_pages'] < threshold, 'num_pages'] = np.nan

book_df = pd.read_csv(this_months_scrape_path)
# book_df = pd.read_csv('data/01-2025_goodreads_scraped.csv')
df = goodreads_export.merge(book_df, on='book_id')

# Drop competing columns
df['author'] = df['author_x'].fillna(df['author_y'])
df['num_pages'] = df['num_pages_x'].fillna(df['num_pages_y'])
df['year'] = df['year_x'].fillna(df['year_y'])
df.drop(columns=['author_x', 'author_y', 'num_pages_x', 'num_pages_y', 'year_x', 'year_y'], inplace=True)

df['genres'] = df['genres'].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else [])
df['year'] = df['year'].fillna(df['year'].mean()).round().astype(int)
df['num_pages'] = df['num_pages'].fillna(df['num_pages'].mean()).round().astype(int)
df['num_reviews'] = df['num_reviews'].fillna(0).round().astype(int)
df['my_rating'] = df['my_rating'].replace(0,np.nan)

df['age'] = int(datetime.now().strftime('%Y')) - df['year']
df['average_rating'] = ((df['5 stars'] * 5) + (df['4 stars'] * 4) + (df['3 stars'] * 3) + (df['2 stars'] * 2) + df['1 star']) / df['num_ratings']
df = df[['book_id', 'title', 'author', 'year', 'age', 'series', 'num_pages', 'genres', 'num_ratings', 'num_reviews', 'my_rating', 'average_rating', '5 stars', '4 stars', '3 stars', '2 stars', '1 star']]

In [ ]:
def fit_quadratic(row):
    x = np.array([1, 2, 3, 4, 5])
    a, b, c = np.polyfit(x, row, 2)
    return pd.Series([a, b, c])

# Calculating quadrdic modeling coefficients
df['1_star_percentage'] = df['1 star'] / df['num_ratings']
df['2_star_percentage'] = df['2 stars'] / df['num_ratings']
df['3_star_percentage'] = df['3 stars'] / df['num_ratings']
df['4_star_percentage'] = df['4 stars'] / df['num_ratings']
df['5_star_percentage'] = df['5 stars'] / df['num_ratings']
coefficients = df[['1_star_percentage','2_star_percentage','3_star_percentage','4_star_percentage','5_star_percentage']].apply(fit_quadratic, axis=1)
df['a'], df['b'], df['c'] = coefficients[0], coefficients[1], coefficients[2]

# Pre-processing columns for rankings
df['num_ratings_ln'] = np.log1p(df['num_ratings'])
df['num_pages_ln'] = np.log1p(df['num_pages'])
df['2a_shifted'] = df['a'] - df['a'].min()
df['2a_shifted'] = df['2a_shifted'] * (1 / df['2a_shifted'].max()) + 1
df['b_shifted'] = df['b'] - df['b'].min()
df['b_shifted'] = df['b_shifted'] * (1 / df['b_shifted'].max()) + 1
df['c_shifted'] = df['c'] - df['c'].min()
df['c_shifted'] = df['c_shifted'] * (1 / df['c_shifted'].max()) + 1

# Types of rankings
df['num_adjusted_rating'] = df['average_rating'] - (df['average_rating'] - df['average_rating'].mean()) / df['num_ratings_ln']
df['coeff_2a_rating'] = (df['num_adjusted_rating'] * df['2a_shifted'])
df['coeff_b_rating'] = (df['num_adjusted_rating']) / (df['b_shifted'])
df['coeff_c_rating'] = (df['num_adjusted_rating'] * df['c_shifted'])
df['joined_rating'] = (df['num_adjusted_rating'] * df['c_shifted'] * df['2a_shifted']) / df['b_shifted']
df['final_rating'] = df['joined_rating'] - (df['joined_rating'] - df['joined_rating'].mean()) / df['num_ratings_ln']

df['num_adjusted_page_rating'] = df['num_adjusted_rating'] / (df['num_pages_ln'])
df['coeff_2a_page_rating'] = df['coeff_2a_rating'] / df['num_pages_ln']
df['coeff_b_page_rating'] = df['coeff_b_rating'] / df['num_pages_ln']
df['coeff_c_page_rating'] = df['coeff_c_rating'] / df['num_pages_ln']
df['joined_page_rating'] = df['joined_rating'] / df['num_pages_ln']
df['final_page_rating'] = df['joined_page_rating'] - (df['joined_page_rating'] - df['joined_page_rating'].mean()) / df['num_ratings_ln']

In [ ]:
numeric_cols = ['age', 'num_pages', 'num_pages_ln', 'num_ratings', 'num_ratings_ln', 'num_reviews', 'my_rating', 'average_rating', '1 star', '2 stars', '3 stars', '4 stars', '5 stars', '1_star_percentage', '2_star_percentage', '3_star_percentage', '4_star_percentage', '5_star_percentage', 'a', 'b', 'c', 'num_adjusted_rating', 'coeff_2a_rating', 'coeff_b_rating', 'coeff_c_rating', 'joined_rating', 'final_rating', 'num_adjusted_page_rating', 'coeff_2a_page_rating', 'coeff_b_page_rating', 'coeff_c_page_rating', 'joined_page_rating', 'final_page_rating']
corr_df= df[numeric_cols].corr()

plt.figure(figsize=(20, 15)) 
sns.heatmap(corr_df, annot=True, cmap='coolwarm', linewidths=0.5) 
plt.title('Correlation Heatmap') 
plt.show()

In [ ]:
fresh = df.sort_values(by='final_page_rating', ascending=False).reset_index().drop('index', axis=1)
fresh = fresh[fresh['my_rating'].isna()]
fresh[['Fiction' in genre_list for genre_list in fresh['genres']]] # Fiction, Nonfiction, Memoir, Classics, History, Politics, Philosophy, Business

In [ ]:
test = pd.read_csv('books_data.csv')
test#['author'].iloc[]